# 3. Clusterização - Base Unificada
Aqui a base unificada será padronizada e clusterizada


In [ ]:
import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import scipy.cluster.hierarchy as sch
#import scipy.stats as stats
from scipy.stats import zscore
from scipy.spatial.distance import pdist
#from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import pingouin as pg
import plotly.express as px 
import plotly.io as pio
from scipy.cluster.hierarchy import fcluster
pio.renderers.default='browser'
from sklearn.metrics import silhouette_score

### Clusterização 1

In [ ]:
base_cluster_completa = pd.read_csv('base_clusterizacao_2023.csv')
base_cluster_completa

In [ ]:
base_cluster_completa.columns

In [4]:
base_cluster_completa[[ 'taxa_aprovacao_em', 'nota_matematica_em', 'nota_portugues_em',
       'nota_media_padronizada_em', 'ideb_em', 'taxa_aprovacao_af',
       'nota_matematica_af', 'nota_portugues_af', 'nota_media_padronizada_af',
       'ideb_af']].describe().to_excel('describe_performance.xlsx')

In [ ]:

# Colunas quantitativas com valores numéricos padronizados
base_cluster_padronizada = base_cluster_completa[[ 'tt_dias_falta_medica_normalizado', 'tt_dias_falta_just_normalizado',
       'tt_dias_falta_injust_normalizado', 'tt_dias_lic_premio_normalizado',
       'tt_dias_lic_gestante_normalizado', 'tt_dias_lic_acid_trab_normalizado',
       'tt_dias_lic_inter_partic_normalizado', 'total_faltas_normalizado',
       'taxa_aprovacao_em', 'nota_matematica_em', 'nota_portugues_em',
       'nota_media_padronizada_em', 'ideb_em', 'taxa_aprovacao_af',
       'nota_matematica_af', 'nota_portugues_af', 'nota_media_padronizada_af',
       'ideb_af']].apply(zscore,ddof=1)

base_cluster_padronizada.head()

#### Elbow

In [ ]:

elbow = []
K = range(1,10) # ponto de parada pode ser parametrizado manualmente
for k in K:
    kmeanElbow = KMeans(n_clusters=k, init='random', random_state=100).fit(base_cluster_padronizada)
    elbow.append(kmeanElbow.inertia_)
    
plt.figure(figsize=(16,8))
plt.plot(K, elbow, marker='o')
plt.xlabel('Nº Clusters', fontsize=16)
plt.xticks(range(1,10)) # ajustar range
plt.ylabel('WCSS', fontsize=16)
plt.savefig('elbow_primeira_clusterizacao.png', dpi=600)
plt.show()

#### Silhueta

In [ ]:
silhueta = []
I = range(2,10) # ponto de parada pode ser parametrizado manualmente
for i in I: 
    kmeansSil = KMeans(n_clusters=i, init='random', random_state=100).fit(base_cluster_padronizada)
    silhueta.append(silhouette_score(base_cluster_padronizada, kmeansSil.labels_))

plt.figure(figsize=(16,8))
plt.plot(range(2, 10), silhueta, color = 'purple', marker='o')
plt.xlabel('Nº Clusters', fontsize=16)
plt.ylabel('Silhueta Média', fontsize=16)
#plt.title('Método da Silhueta', fontsize=16)
plt.axvline(x = silhueta.index(max(silhueta))+2, linestyle = 'dotted', color = 'red') 
plt.savefig('silhueta_primeira_clusterizacao.png', dpi=600)
plt.show()

#### Clusterização k-means

In [ ]:
#%% Cluster Não Hierárquico K-means

# Considerando que identificamos 3 possíveis clusters na análise hierárquica

kmeans = KMeans(n_clusters=2, init='random', random_state=100,n_init='auto').fit(base_cluster_padronizada)

# Gerando a variável para identificarmos os clusters gerados

kmeans_clusters = kmeans.labels_
base_cluster_completa['cluster_kmeans'] = kmeans_clusters
base_cluster_padronizada['cluster_kmeans'] = kmeans_clusters


base_cluster_completa


#### Observando as características dos Clusters

In [ ]:
import seaborn as sns

# Set the style for the plots
sns.set_theme(style="whitegrid")

numeric_columns = [
     'total_servidores',
       'tt_dias_falta_medica_normalizado', 'tt_dias_falta_just_normalizado',
       'tt_dias_falta_injust_normalizado', 'tt_dias_lic_premio_normalizado',
       'tt_dias_lic_gestante_normalizado', 'tt_dias_lic_acid_trab_normalizado',
       'tt_dias_lic_inter_partic_normalizado', 'total_faltas_normalizado'
]

# Create subplots for each numeric column
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(20, 20))
axes = axes.flatten()

# Loop through the numeric columns and create a bar chart for each
for i, column in enumerate(numeric_columns):
    # Calculate mean values grouped by 'cluster_kmeans'
    mean_values = base_cluster_completa.groupby('cluster_kmeans')[column].mean().reset_index()
    
    # Create barplot
    sns.barplot(data=mean_values, x='cluster_kmeans', y=column, ax=axes[i])
    
    # Set titles and labels
    axes[i].set_title(f'Média {column} por Cluster KMeans')
    axes[i].set_xlabel('Cluster')
    axes[i].set_ylabel(f'Média {column}')

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

numeric_columns = [ 'taxa_aprovacao_em','nota_matematica_em', 'nota_portugues_em',
       'nota_media_padronizada_em', 'ideb_em', 'taxa_aprovacao_af',
       'nota_matematica_af', 'nota_portugues_af', 'nota_media_padronizada_af',
       'ideb_af'
]

# Create subplots for each numeric column
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(20, 20))
axes = axes.flatten()

# Loop through the numeric columns and create a bar chart for each
for i, column in enumerate(numeric_columns):
    # Calculate mean values grouped by 'cluster_kmeans'
    mean_values = base_cluster_completa.groupby('cluster_kmeans')[column].mean().reset_index()
    
    # Create barplot
    sns.barplot(data=mean_values, x='cluster_kmeans', y=column, ax=axes[i])
    
    # Set dynamic y-axis limits to "zoom in"
    min_value = mean_values[column].min()
    max_value = mean_values[column].max()
    axes[i].set_ylim(min_value - 0.1 * (max_value - min_value), max_value + 0.1 * (max_value - min_value))
    
  # Set titles and labels
    axes[i].set_title(f'Média {column} por Cluster KMeans', fontsize=18)
    axes[i].set_xlabel('Cluster', fontsize=18)
    axes[i].set_ylabel(f'Valor Médio', fontsize=18)

# Adjust layout for better spacing
plt.tight_layout()
plt.show()


#### Estatísticas F - Determinando se todas as variáveis auxiliaram na formação dos clusters

In [ ]:
lista_variaveis = list(base_cluster_padronizada.columns)
#Lista de dicionários com informações sobre as variáveis e seus respectivos testes
lista_testes = []

#Criando uma lista de testes F para utilização posterior
for variavel in lista_variaveis:
    if variavel != 'cluster_kmeans':
        print(variavel)
        teste_f = pg.anova(dv=variavel, 
            between='cluster_kmeans', 
            data=base_cluster_padronizada,
            detailed=True).T
        teste_dict = {'variavel':variavel,'resultados_teste':teste_f}
        lista_testes.append(teste_dict)

In [19]:
#Estatísticas Descritivas - Cluster 0 
base_cluster_completa.query("cluster_kmeans == 0")[['total_faltas_normalizado','ideb_em','ideb_af']].describe().T[['min','25%','50%','75%','max']].to_excel('descritivas_0.xlsx')

In [12]:
#Estatísticas Descritivas - Cluster 1
base_cluster_completa.query("cluster_kmeans == 1")[['total_faltas_normalizado','ideb_em','ideb_af']].describe().T[['min','25%','50%','75%','max']].to_excel('descritivas_1.xlsx')

In [ ]:

#Estatisticas descritivas agregadas por cluster
descritivas_por_cluster = base_cluster_completa[[
       'tt_dias_falta_medica_normalizado', 'tt_dias_falta_just_normalizado',
       'tt_dias_falta_injust_normalizado', 'tt_dias_lic_premio_normalizado',
       'tt_dias_lic_gestante_normalizado', 'tt_dias_lic_acid_trab_normalizado',
       'tt_dias_lic_inter_partic_normalizado', 'total_faltas_normalizado',
       'taxa_aprovacao_em', 'nota_matematica_em', 'nota_portugues_em',
       'nota_media_padronizada_em', 'ideb_em', 'taxa_aprovacao_af',
       'nota_matematica_af', 'nota_portugues_af', 'nota_media_padronizada_af',
       'ideb_af','cluster_kmeans']].groupby('cluster_kmeans').mean().reset_index()

descritivas_por_cluster.drop(columns=['cluster_kmeans']).T

In [ ]:
#Contagem de Municípios por cluster
cluster_counts = base_cluster_completa['cluster_kmeans'].value_counts()


plt.figure(figsize=(3, 3))
plt.pie(cluster_counts, 
        labels=[f'Cluster {cluster} ({count} - {count/sum(cluster_counts)*100:.2f}%)' 
                for cluster, count in cluster_counts.items()], 
        autopct='%1.1f%%', startangle=90)


plt.axis('equal')


plt.savefig('distribuicao_primeira_clusterizacao.png', dpi=600)
plt.show()

In [27]:
import pandas as pd
#Salvando o resultado dos testes em uma tabela do excel

rows = []


for entry in lista_testes:
    variable_name = entry['variavel']
    test_results = entry['resultados_teste']
    f_stat = test_results.loc['F'][0]  # F-statistic from column 0
    p_value = test_results.loc['p-unc'][0]  # p-value from column 0
    rows.append([variable_name, f_stat, p_value])

# Create a dataframe from the extracted data
result_df = pd.DataFrame(rows, columns=['Test Name', 'F Statistic', 'p-value'])

# Display the resulting dataframe
result_df.to_excel('estatisticas_f.xlsx',index = False)



### Clusterização 2 - Cluster com mais faltas
Com o objetivo de entender os padrões das faltas em relação aos municípios com mais faltas e pior desempenho, vamos fazer uma segunda clusterização dentro desse cluster 

In [ ]:
base_cluster_completa.query("cluster_kmeans == 0").query("tt_dias_lic_acid_trab_normalizado > 0")

In [ ]:
#Vamos eliminar  tt_dias_lic_acid_trab_normalizado, pois possui poucos valores que possuem algum valor
# O unico valor pertence a cidade bom jesus dos perdoes
base_cluster_completa.query("cluster_kmeans == 0").describe()

In [ ]:
#Cluster + faltas - performance
#Como essa é uma análise mais focada no perfil de faltas, vamos eliminar a coluna relacionada ao total de faltas, como estamos focando mais no caráter das faltas, vamos manter apenas o ideb do ensino médio e do fundamental

cluster_faltas_padronizado = base_cluster_padronizada.query("cluster_kmeans == 0").drop(columns = ['tt_dias_lic_acid_trab_normalizado','total_faltas_normalizado','cluster_kmeans']).reset_index(drop = True)
cluster_faltas = base_cluster_completa.query("cluster_kmeans == 0").drop(columns = ['tt_dias_lic_acid_trab_normalizado','total_faltas_normalizado','cluster_kmeans']).reset_index(drop = True)

# Verificando a quantidade final de colunas
cluster_faltas.columns

#### Elbow

In [ ]:

elbow = []
K = range(1,9) # ponto de parada pode ser parametrizado manualmente
for k in K:
    kmeanElbow_cluster_zero = KMeans(n_clusters=k, init='random', random_state=100).fit(cluster_faltas_padronizado)
    elbow.append(kmeanElbow_cluster_zero.inertia_)
    
plt.figure(figsize=(16,8))
plt.plot(K, elbow, marker='o')
plt.xlabel('Nº Clusters', fontsize=16)
plt.xticks(range(1,9)) # ajustar range
plt.ylabel('WCSS', fontsize=16)
plt.savefig('elbow_primeira_clusterizacao.png', dpi=600)
plt.show()

A análise indica que a quantidade ideal de clusters é 2

#### Silhueta

In [ ]:
silhueta = []
I = range(2,10) # ponto de parada pode ser parametrizado manualmente
for i in I: 
    kmeansSil_cluster_zero = KMeans(n_clusters=i, init='random', random_state=100).fit(cluster_faltas_padronizado)
    silhueta.append(silhouette_score(cluster_faltas_padronizado, kmeansSil_cluster_zero.labels_))

plt.figure(figsize=(16,8))
plt.plot(range(2, 10), silhueta, color = 'purple', marker='o')
plt.xlabel('Nº Clusters', fontsize=16)
plt.ylabel('Silhueta Média', fontsize=16)
#plt.title('Método da Silhueta', fontsize=16)
plt.axvline(x = silhueta.index(max(silhueta))+2, linestyle = 'dotted', color = 'red') 
plt.savefig('subagrupamento_0_silhueta.png', dpi=600)
plt.show()

A quantidade ideal de clusters é 2

#### Clusterização - 2 Clusters

In [ ]:
# As análises indicam que o número ideal de clusters é = a 2, vamos então  prosseguir para a clusterização

# Considerando que identificamos 3 possíveis clusters na análise hierárquica

kmeans_cluster_zero = KMeans(n_clusters=2, init='random', random_state=100,n_init='auto').fit(cluster_faltas_padronizado)

# Gerando a variável para identificarmos os clusters gerados

kmeans_clusters_padronizado = kmeans_cluster_zero.labels_
cluster_faltas['subcluster_kmeans'] = kmeans_clusters_padronizado
cluster_faltas_padronizado['subcluster_kmeans'] = kmeans_clusters_padronizado

cluster_faltas


In [ ]:
cluster_counts_padronizado = cluster_faltas['subcluster_kmeans'].value_counts()

#Distribuição dos municípios da segunda clusterização
plt.figure(figsize=(3, 3))
plt.pie(cluster_counts_padronizado, 
        labels=[f'Cluster {cluster} ({count} - {count/sum(cluster_counts_padronizado)*100:.2f}%)' 
                for cluster, count in cluster_counts_padronizado.items()], 
        autopct='%1.1f%%', startangle=90)


plt.axis('equal')


plt.savefig('distribuicao_subagrupamento.png', dpi=600)
plt.show()

In [ ]:
#Plotando as médias em paineis 
import seaborn as sns

sns.set_theme(style="whitegrid")

numeric_columns = [
       'tt_dias_falta_medica_normalizado', 'tt_dias_falta_just_normalizado'
]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
axes = axes.flatten()

for i, column in enumerate(numeric_columns):
    mean_values = cluster_faltas.groupby('subcluster_kmeans')[column].mean().reset_index()
    
    sns.barplot(data=mean_values, x='subcluster_kmeans', y=column, ax=axes[i])
    axes[i].set_title(f'Média {column}')
    axes[i].set_xlabel('Cluster')
    axes[i].set_ylabel(f'Média {column}')

plt.savefig('caracteristicas_subagrupamentos_assiduidade_1.png', dpi=600)

plt.tight_layout()
plt.show()

In [ ]:
#Plotando as médias em paineis 
import seaborn as sns


sns.set_theme(style="whitegrid")

numeric_columns = [

       'tt_dias_falta_injust_normalizado', 'tt_dias_lic_premio_normalizado'
]


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
axes = axes.flatten()

for i, column in enumerate(numeric_columns):
    mean_values = cluster_faltas.groupby('subcluster_kmeans')[column].mean().reset_index()
    
    sns.barplot(data=mean_values, x='subcluster_kmeans', y=column, ax=axes[i])
    axes[i].set_title(f'Média {column}')
    axes[i].set_xlabel('Cluster')
    axes[i].set_ylabel(f'Média {column}')


plt.savefig('caracteristicas_subagrupamentos_assiduidade_2.png', dpi=600)

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns

#Plotando as médias em paineis 
sns.set_theme(style="whitegrid")

numeric_columns = [
       'tt_dias_lic_gestante_normalizado',
       'tt_dias_lic_inter_partic_normalizado'
]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
axes = axes.flatten()


for i, column in enumerate(numeric_columns):
    mean_values = cluster_faltas.groupby('subcluster_kmeans')[column].mean().reset_index()
    
    sns.barplot(data=mean_values, x='subcluster_kmeans', y=column, ax=axes[i])
    
    axes[i].set_title(f'Média {column}')
    axes[i].set_xlabel('Cluster')
    axes[i].set_ylabel(f'Média {column}')

plt.savefig('caracteristicas_subagrupamentos_assiduidade_3.png', dpi=600)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#Plotando as médias em paineis 
numeric_columns = [ 
       'ideb_em',
       'ideb_af'
]

# Create subplots for each numeric column
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
axes = axes.flatten()

# Loop through the numeric columns and create a bar chart for each
for i, column in enumerate(numeric_columns):
    # Calculate mean values grouped by 'cluster_kmeans'
    mean_values = cluster_faltas.groupby('subcluster_kmeans')[column].mean().reset_index()
    
    # Create barplot
    sns.barplot(data=mean_values, x='subcluster_kmeans', y=column, ax=axes[i])
    
    # Set dynamic y-axis limits to "zoom in"
    min_value = mean_values[column].min()
    max_value = mean_values[column].max()
    axes[i].set_ylim(min_value - 0.1 * (max_value - min_value), max_value + 0.1 * (max_value - min_value))
    
  # Set titles and labels
    axes[i].set_title(f'Média {column} por Cluster KMeans', fontsize=18)
    axes[i].set_xlabel('Cluster', fontsize=18)
    axes[i].set_ylabel(f'Média {column}', fontsize=18)

# Adjust layout for better spacing
plt.savefig('caracteristicas_subagrupamentos_performance.png', dpi=600)

plt.tight_layout()
plt.show()


In [ ]:


#Plotando os dados em paineis
sns.set_theme(style="whitegrid")

numeric_columns = [
     'total_servidores',
       'tt_dias_falta_medica_normalizado', 'tt_dias_falta_just_normalizado'
]


fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
axes = axes.flatten()


for i, column in enumerate(numeric_columns):
    mean_values = base_cluster_completa.groupby('cluster_kmeans')[column].mean().reset_index()
    
    sns.barplot(data=mean_values, x='cluster_kmeans', y=column, ax=axes[i])
    
    axes[i].set_title(f'Média {column} por Cluster KMeans')
    axes[i].set_xlabel('Cluster')
    axes[i].set_ylabel(f'Valor médio')




plt.tight_layout()

plt.savefig('painel_cluster_1.png', dpi=600)
plt.show()

In [ ]:
#Plotando os dados em paineis


sns.set_theme(style="whitegrid")

numeric_columns = [
       'tt_dias_falta_injust_normalizado', 'tt_dias_lic_premio_normalizado',
       'tt_dias_lic_gestante_normalizado'
]


fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
axes = axes.flatten()


for i, column in enumerate(numeric_columns):
    mean_values = base_cluster_completa.groupby('cluster_kmeans')[column].mean().reset_index()
    

    sns.barplot(data=mean_values, x='cluster_kmeans', y=column, ax=axes[i])
    

    axes[i].set_title(f'Média {column} por Cluster KMeans')
    axes[i].set_xlabel('Cluster')
    axes[i].set_ylabel(f'Valor médio')


plt.tight_layout()
plt.savefig('painel_cluster_2.png', dpi=600)
plt.show()

In [ ]:


#Plotando os dados em paineis
sns.set_theme(style="whitegrid")

numeric_columns = [
  'tt_dias_lic_acid_trab_normalizado',
       'tt_dias_lic_inter_partic_normalizado', 'total_faltas_normalizado'
]


fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
axes = axes.flatten()

for i, column in enumerate(numeric_columns):
    mean_values = base_cluster_completa.groupby('cluster_kmeans')[column].mean().reset_index()
    

    sns.barplot(data=mean_values, x='cluster_kmeans', y=column, ax=axes[i])
    

    axes[i].set_title(f'Média {column} por Cluster KMeans')
    axes[i].set_xlabel('Cluster')
    axes[i].set_ylabel(f'Valor médio')


plt.tight_layout()
plt.savefig('painel_cluster_3.png', dpi=600)
plt.show()

In [ ]:
#Plotando os dados em paineis

numeric_columns = [ 'taxa_aprovacao_em','nota_matematica_em', 'nota_portugues_em',
       'nota_media_padronizada_em'
]


fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 15))
axes = axes.flatten()


for i, column in enumerate(numeric_columns):
    mean_values = base_cluster_completa.groupby('cluster_kmeans')[column].mean().reset_index()
    
    sns.barplot(data=mean_values, x='cluster_kmeans', y=column, ax=axes[i])
    
    min_value = mean_values[column].min()
    max_value = mean_values[column].max()
    axes[i].set_ylim(min_value - 0.1 * (max_value - min_value), max_value + 0.1 * (max_value - min_value))

    axes[i].set_title(f'Média {column} por Cluster KMeans', fontsize=18)
    axes[i].set_xlabel('Cluster', fontsize=18)
    axes[i].set_ylabel(f'Média {column}', fontsize=18)


plt.tight_layout()
plt.savefig('painel_performance_1.png', dpi=600)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#Plotando os dados em paineis

numeric_columns = [ 'taxa_aprovacao_em','nota_matematica_em', 'nota_portugues_em',
       'nota_media_padronizada_em', 'ideb_em', 'taxa_aprovacao_af',
       'nota_matematica_af', 'nota_portugues_af', 'nota_media_padronizada_af',
       'ideb_af'
]


fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(20, 20))
axes = axes.flatten()

for i, column in enumerate(numeric_columns):
    mean_values = base_cluster_completa.groupby('cluster_kmeans')[column].mean().reset_index()
    

    sns.barplot(data=mean_values, x='cluster_kmeans', y=column, ax=axes[i])
    

    min_value = mean_values[column].min()
    max_value = mean_values[column].max()
    axes[i].set_ylim(min_value - 0.1 * (max_value - min_value), max_value + 0.1 * (max_value - min_value))
    

    axes[i].set_title(f'Média {column} por Cluster KMeans', fontsize=18)
    axes[i].set_xlabel('Cluster', fontsize=18)
    axes[i].set_ylabel(f'Média {column}', fontsize=18)


plt.tight_layout()
plt.savefig('painel_performance_1.png', dpi=600)
plt.show()
